## 导入库

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import annotations

# ── 标准库 ──────────────────────────────────────────────────────────────────
import os
import time
from pathlib import Path
from collections import defaultdict
from datetime import datetime

# ── 第三方 ──────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import polars as pl

import torch
import torch.backends.cudnn as cudnn
import lightning as L
import lightning.pytorch as lp
from torch.utils.data import DataLoader

from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import DeviceStatsMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline
from pytorch_forecasting.metrics import MAE, RMSE
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_forecasting.data import TorchNormalizer, GroupNormalizer


# 你的工程工具
from pipeline.io import cfg, P, fs, storage_options, ensure_dir_local
from pipeline.stream_input_local import ShardedBatchStream  # 使用下方给你的新版类
from pipeline.wr2 import WR2

# ---- 性能/兼容开关（仅一次）----
os.environ.setdefault("POLARS_MAX_THREADS", str(max(1, os.cpu_count() // 2)))
pl.enable_string_cache()
cudnn.benchmark = True
torch.set_float32_matmul_precision("high")

import time as _t


import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths

def _now() -> str:
    return _t.strftime("%Y-%m-%d %H:%M:%S")
print(f"[{_now()}] imports ok")

## 定义工具函数

In [ ]:
# ───────────────────────────────────────────────────────────────────────────
# 滑动窗划分
def make_sliding_cv_by_days(all_days: np.ndarray, *, n_splits: int, gap_days: int, train_to_val: int):
    all_days = np.asarray(all_days).ravel()
    K, R, G = n_splits, train_to_val, gap_days
    usable = len(all_days) - G
    if usable <= 0 or K <= 0 or R <= 0:
        return []
    V_base, rem = divmod(usable, R + K)
    if V_base <= 0:
        return []
    T = R * V_base
    v_lens = [V_base + 1 if i < rem else V_base for i in range(K)]
    folds, v_lo = [], T + G
    for V_i in v_lens:
        v_hi, tr_hi, tr_lo = v_lo + V_i, v_lo - G, v_lo - G - T
        if tr_lo < 0 or v_hi > len(all_days):
            break
        folds.append((all_days[tr_lo:tr_hi], all_days[v_lo:v_hi]))
        v_lo = v_hi
    return folds


## 初始化参数

In [ ]:


# ========== 1) 统一配置 ==========
G_SYM, G_DATE, G_TIME = cfg["keys"]          # e.g. ("symbol_id","date_id","time_id")
TARGET_COL = cfg["target"]                   # e.g. "responder_6"
WEIGHT_COL = cfg["weight"]                   # 允许为 None

# 已知时间特征（给 encoder+decoder）
TIME_FEATURES = ["time_bucket"]

# 连续特征（示例）
#BASIC_FEATURES = [f"feature_{i:02d}" for i in range(79)]
BASIC_FEATURES = [
    "feature_01",
    "feature_04",
    "feature_05",
    "feature_06",
    "feature_07",
    "feature_08",
    "feature_16",
    "feature_20",
    "feature_22",
    "feature_24",
    "feature_27",
    "feature_36",
    "feature_37",
    "feature_38",
    "feature_58",
    "feature_69",
]
BASIC_RESPONDERS = [f"responder_{i}" for i in range(0, 9)]

RAW_FEATURES = BASIC_FEATURES 

# 训练 & CV 超参
N_SPLITS     = 1
GAP_DAYS     = 5
TRAIN_TO_VAL = 8
ENC_LEN      = 50
DEC_LEN      = 1
PRED_LEN     = DEC_LEN
BATCH_SIZE   = 256
LR           = 3e-4
HIDDEN       = 32
HEADS        = 2
DROPOUT      = 0.2
MAX_EPOCHS   = 30
CHUNK_DAYS   = 30

# 数据路径
PANEL_DIR_AZ   = P("az", cfg["paths"].get("panel_shards", "panel_shards"))
TFT_LOCAL_ROOT = P("local", "tft")

# 目录准备
ensure_dir_local(TFT_LOCAL_ROOT)
LOCAL_CLEAN_DIR = f"{TFT_LOCAL_ROOT}/clean"; ensure_dir_local(LOCAL_CLEAN_DIR)
CKPTS_DIR = Path(TFT_LOCAL_ROOT) / "ckpts"; ensure_dir_local(CKPTS_DIR.as_posix())
LOGS_DIR  = Path(TFT_LOCAL_ROOT) / "logs";  ensure_dir_local(LOGS_DIR.as_posix())

print("[config] ready")


## 数据导入

In [ ]:
#data_paths = fs.glob(f"{PANEL_DIR_AZ}/*.parquet")
#data_paths = [p if p.startswith("az://") else f"az://{p}" for p in data_paths]

data_start_date = 1500
data_end_date = 1580


data_paths = fs.glob("az://jackson/js_exp/exp/v1/clean_shards/*.parquet")
data_paths =[f"az://{p}" for p in data_paths]

lf_data = pl.scan_parquet(data_paths, storage_options=storage_options).filter(pl.col(G_DATE).is_between(data_start_date, data_end_date, closed="both"))
lf_data = lf_data.sort([G_SYM, G_DATE, G_TIME])

In [ ]:
lf_data.limit(5).collect()

## 数据处理

### 添加全局时间序列号

In [ ]:
lf_grid = (
    lf_data.select([G_DATE, G_TIME]).unique()
        .sort([G_DATE, G_TIME])
        .with_row_index("time_idx")
        .with_columns(pl.col("time_idx").cast(pl.Int64))
)

NEED_COLS = list(dict.fromkeys([G_SYM, G_DATE, G_TIME, WEIGHT_COL, TARGET_COL] + TIME_FEATURES + RAW_FEATURES + BASIC_RESPONDERS))
lf_with_time_idx = (
    lf_data.join(lf_grid, on=[G_DATE, G_TIME], how="left")
        .select(["time_idx"] + NEED_COLS)
        .sort([G_SYM, "time_idx"])
)

### 简单的特征工程

*完成初步验证后就删掉，用回由LGBM选中的特征

In [ ]:
# 添加每天的时间正余弦

T = 968
twopi_over_T = np.float32(2.0*np.pi) / T
ti_f = pl.col(G_TIME).cast(pl.Float32)

# 基表筛列 + 时间特征
lf_with_sincos = (
    lf_with_time_idx
    .with_columns([
        ti_f.alias("time_pos"),
        (ti_f * pl.lit(twopi_over_T, dtype=pl.Float32)).alias("_phase_"),
        ])
    .with_columns([
        pl.col("_phase_").sin().cast(pl.Float32).alias("time_sin"),
        pl.col("_phase_").cos().cast(pl.Float32).alias("time_cos"),
        ])
    .drop(["_phase_"])
).sort([G_SYM, "time_idx"])


In [ ]:
lf_with_sincos.limit(5).collect()   

In [ ]:

#先做responders日级摘要

# 日频统计聚合
aggs = []
for r in BASIC_RESPONDERS:
    aggs.extend([
        pl.col(r).min().alias(f"{r}_prevday_min"),
        pl.col(r).max().alias(f"{r}_prevday_max"),
        pl.col(r).mean().alias(f"{r}_prevday_mean"),
        pl.col(r).std(ddof=0).alias(f"{r}_prevday_std"),
        pl.col(r).first().alias(f"{r}_prevday_open"),
        pl.col(r).last().alias(f"{r}_prevday_close"),
        (pl.col(r).last() / (pl.col(r).first() + 1e-8) - 1).alias(f"{r}_prevday_chg"),
    ])

lf_daily = (
    lf_with_sincos
    .group_by([G_SYM, G_DATE])
    .agg(aggs)
    .sort([G_SYM, G_DATE])
)
lf_daily.limit(5).collect()


In [ ]:
# responders ema
R_EMA_SPANS = [3, 5, 7]

# 取列名（LazyFrame）
schema_cols = lf_daily.collect_schema().names()  # 或 lf_daily.schema.names()
r_cols = [c for c in schema_cols if c.startswith("responder_")]

In [ ]:
r_ewm_exprs = []
for c in r_cols:
    for s in R_EMA_SPANS:
        r_ewm_exprs.append(
            pl.col(c)
            .ewm_mean(span=int(s), adjust=False, ignore_nulls=True, min_periods=1)
            .over(G_SYM)
            .cast(pl.Float32)
            .alias(f"{c}__ewm{s}")
        )

lf_daily = lf_daily.with_columns(r_ewm_exprs)


In [ ]:
# 判断日频表的每个组的日期是否连续
lf_daily_with_diff =lf_daily.with_columns(
    [pl.col(G_DATE).diff().over(G_SYM).alias("date_diff"),
    ]
)

# 关键，shift为前一日
shifted_cols = [c for c in lf_daily.collect_schema().names() if c not in (G_SYM, G_DATE, "date_diff")]
lf_prevday_daily = (
    lf_daily_with_diff
    .with_columns([
        pl.when(pl.col("date_diff") <= 5)
        .then(pl.col(c).shift(1).over([G_SYM]))  # ← 按组上一天
        .otherwise(pl.lit(None))                # 超过阈值置空
        .alias(c)                               # 覆盖原列（你当前的语义）
        for c in shifted_cols
    ])
    .drop("date_diff")
)

# 回拼到 tick 级
lf_extended = (
    lf_with_sincos.join(lf_prevday_daily, on=[G_SYM, G_DATE], how="left")
).sort([G_SYM, "time_idx"])
to_drop = [c for c in BASIC_RESPONDERS if c != TARGET_COL]
lf_extended = lf_extended.drop(pl.col(to_drop))


In [ ]:
pl.Config.set_tbl_cols(-1)      # 显示所有列
pl.Config.set_tbl_rows(-1)      # 显示所有行
lf_extended.select(pl.all().is_null().sum()).collect()

In [ ]:
# 添加特征量的ewa

feature_ewm_exprs = []
feature_ewm_cols = []
for c in BASIC_FEATURES:
    for s in R_EMA_SPANS:
        col_name = f"{c}__ewm{s}"
        feature_ewm_cols.append(col_name)
        feature_ewm_exprs.append(
            pl.col(c)
            .ewm_mean(span=int(s), adjust=False, ignore_nulls=True, min_periods=1)
            .over([G_SYM, G_DATE])
            .cast(pl.Float32)
            .alias(col_name)
        )
lf_feature_ewm = lf_with_sincos.with_columns(feature_ewm_exprs)
lf_feature_ewm = lf_feature_ewm.drop(pl.col(to_drop))

In [ ]:
lf_combined = (
    lf_extended
    .join(lf_feature_ewm.select([G_SYM, G_DATE, G_TIME] + feature_ewm_cols), on=[G_SYM, G_DATE, G_TIME], how="left")
).sort([G_SYM, "time_idx"])

In [ ]:
# 过滤掉第一天的数据
lf_ready = lf_combined.filter(pl.col(G_DATE)>1500) #  去掉全是null 的第一天数据
print(f"[{_now()}] lazyframe ready")

In [ ]:
missing_r_cols = [c for c in lf_ready.collect_schema().names() if c.startswith("responder_")]
fill_exprs = [pl.col(c).fill_null(0).cast(pl.Float32).alias(c) for c in missing_r_cols]
    
lf_ready = lf_ready.with_columns(fill_exprs)


In [ ]:
pl.Config.set_tbl_cols(-1)      # 显示所有列
pl.Config.set_tbl_rows(-1)      # 显示所有行
lf_ready.select(pl.all().is_null().sum()).collect() 

## CV 划分

In [ ]:
# ==========  CV 划分 ==========
all_days = (
    lf_ready.select(pl.col(G_DATE)).unique().sort(by=G_DATE)
    .collect(streaming=True).get_column(G_DATE).to_numpy()
)
folds_by_day = make_sliding_cv_by_days(all_days, n_splits=N_SPLITS, gap_days=GAP_DAYS, train_to_val=TRAIN_TO_VAL)
assert len(folds_by_day) > 0, "no CV folds constructed"

## 折内数据标准化处理

In [ ]:
lf_ready.collect_schema().names()

In [ ]:
# 明确要标准化的列
z_cols = [c for c in lf_ready.collect_schema().names() if c not in (G_SYM, G_DATE, G_TIME, "time_idx", "time_pos", "time_sin", "time_cos", "time_bucket", WEIGHT_COL, TARGET_COL)]


# 取第一个 fold 的训练集最后一天，作为统计 z-score 的上界
stats_hi = int(folds_by_day[0][0][-1])
print(f"标准化使用训练集的上限日期 = {stats_hi}")

# ========== 4) 连续特征清洗 + Z-score ==========
inf2null_exprs  = [pl.when(pl.col(c).is_infinite()).then(None).otherwise(pl.col(c)).alias(c) for c in z_cols]
#isna_flag_exprs = [pl.col(c).is_null().cast(pl.Int8).alias(f"{c}__isna") for c in z_cols]

ffill_exprs = [pl.col(c).forward_fill().over(G_SYM).fill_null(0.0).alias(c) for c in z_cols]
lf_clean = (
    lf_ready.with_columns(inf2null_exprs).with_columns(ffill_exprs) #.with_columns(isna_flag_exprs)
)


# 开始计算 z-score
lf_stats_sym = (
    lf_clean.filter(pl.col(G_DATE) <= stats_hi)
            .group_by(G_SYM)
            .agg([pl.col(c).mean().alias(f"mu_{c}") for c in z_cols] +
                [pl.col(c).std(ddof=0).alias(f"std_{c}") for c in z_cols])
)
lf_stats_glb = (
    lf_clean.filter(pl.col(G_DATE) <= stats_hi)
            .select([pl.col(c).mean().alias(f"mu_{c}_glb") for c in z_cols] +
                    [pl.col(c).std(ddof=0).alias(f"std_{c}_glb") for c in z_cols])
)


In [ ]:
lf_z = lf_clean.join(lf_stats_glb, how="cross").join(lf_stats_sym, on=G_SYM, how="left").sort([G_SYM, "time_idx"])

eps = 1e-6
#Z_COLS, NAMARK_COLS = [], [f"{c}__isna" for c in RAW_FEATURES]

Z_COLS = []
for c in z_cols:
    mu_sym, std_sym = f"mu_{c}", f"std_{c}"
    mu_glb, std_glb = f"mu_{c}_glb", f"std_{c}_glb"
    mu_use, std_use = f"{c}_mu_use", f"{c}_std_use"
    z_name = f"{c}_z"

    lf_z = lf_z.with_columns(
        pl.when(pl.col(mu_sym).is_null()).then(pl.col(mu_glb)).otherwise(pl.col(mu_sym)).alias(mu_use),
        pl.when(pl.col(std_sym).is_null() | (pl.col(std_sym) == 0)).then(pl.col(std_glb)).otherwise(pl.col(std_sym)).alias(std_use),
    ).with_columns(
        ((pl.col(c) - pl.col(mu_use)) / (pl.col(std_use) + eps)).alias(z_name)
    ).drop([mu_glb, std_glb, mu_sym, std_sym, mu_use, std_use])

    Z_COLS.append(z_name)

In [ ]:
OUT_COLS = [G_SYM, G_DATE, G_TIME, "time_idx", WEIGHT_COL, TARGET_COL, "time_pos", "time_sin", "time_cos", "time_bucket"] + Z_COLS 
lf_out = lf_z.select(OUT_COLS)

In [ ]:
# 保存lf_out到本地
clean_path_local = P("local", "tft/panel/clean.parquet")
Path(clean_path_local).parent.mkdir(parents=True, exist_ok=True)
lf_out.collect(streaming=True).write_parquet(clean_path_local, compression="zstd")
print(f"[{_now()}] cleaned data saved to {clean_path_local}")


## 训练模型

In [ ]:
# clean 数据导入
clean_path_local = P("local", "tft/panel/clean.parquet")
lf_clean = pl.scan_parquet(clean_path_local)
lf_clean.limit(5).collect()

In [ ]:
KNOWN_REALS = [c for c in lf_clean.collect_schema().names() if c not in (G_SYM, G_DATE, G_TIME, "time_idx", "time_bucket", WEIGHT_COL, TARGET_COL)]

KNOWN_CATEGORIES = ["time_bucket"]

UNSCALE_COLS = KNOWN_REALS

TRAIN_COLS = [c for c in lf_clean.collect_schema().names() if c not in (G_DATE, G_TIME)]

# 定义 identity scalers
identity_scalers = {name: None for name in UNSCALE_COLS}

## try

In [ ]:
# 取第一折先试探一下
best_ckpt_paths, fold_metrics = [], []
fold_id = 0
train_days, val_days = folds_by_day[0]

print(f"[fold {fold_id}] train {train_days[0]}..{train_days[-1]} ({len(train_days)} days), "
    f"val {val_days[0]}..{val_days[-1]} ({len(val_days)} days)")

# 明确日期：
train_start_date = int(train_days[0])
train_end_date   = int(train_days[-1])
val_start_date   = int(val_days[0])
val_end_date     = int(val_days[-1])      


# 提取数据
date_range = (train_start_date, val_end_date)
pdf_data = (
    pl.scan_parquet(clean_path_local)
    .filter(pl.col(G_DATE).between(train_start_date, val_end_date, closed="both"))
    .collect(streaming=True)
    .to_pandas()
    .sort_values([G_SYM, "time_idx"])
)
pdf_data[G_SYM] = pdf_data[G_SYM].astype("str")   

# 明确 indexes:
train_end_idx = pdf_data.loc[pdf_data[G_DATE] == train_end_date, "time_idx"].max()
val_start_idx = pdf_data.loc[pdf_data[G_DATE] == val_start_date, "time_idx"].min()
val_end_idx   = pdf_data.loc[pdf_data[G_DATE] == val_end_date, "time_idx"].max()
assert pd.notna(train_end_idx) and pd.notna(val_start_idx) and pd.notna(val_end_idx), "train/val idx not found"
train_end_idx, val_start_idx, val_end_idx = int(train_end_idx), int(val_start_idx), int(val_end_idx)
print(f"[fold {fold_id}] train idx up to {train_end_idx}, val idx {val_start_idx}..{val_end_idx}")  



In [ ]:
t_data = pdf_data[TRAIN_COLS].copy()

identity_scalers = {name: None for name in UNSCALE_COLS}
base_ds = TimeSeriesDataSet(
    t_data,
    time_idx="time_idx",
    target=TARGET_COL,
    group_ids=[G_SYM],
    weight=WEIGHT_COL,
    max_encoder_length=ENC_LEN, 
    min_encoder_length=ENC_LEN,
    max_prediction_length=PRED_LEN, 
    min_prediction_length=PRED_LEN,
    
    static_categoricals=[G_SYM],
    time_varying_known_reals =KNOWN_REALS,
    time_varying_unknown_reals=UNKNOWN_REALS,
    
    lags=None,
    categorical_encoders={G_SYM: NaNLabelEncoder(add_nan=True)},
    add_relative_time_idx=False,
    add_target_scales=False,
    add_encoder_length=False,
    allow_missing_timesteps=True,
    target_normalizer=TorchNormalizer(method="identity"),
    scalers=identity_scalers,
)

In [ ]:
# 划分训练集，验证集
train_ds = base_ds.filter(
    lambda idx: (
        idx.time_idx_last <= train_end_idx
    ),
    copy=True
)

val_ds = base_ds.filter(
    lambda idx: (
        (idx.time_idx_first_prediction >= val_start_idx) &
        
        (idx.time_idx_last <= val_end_idx)
    ),
    copy=True
)

In [ ]:
# 数据集加载

train_loader = train_ds.to_dataloader(
    train=True, 
    batch_size=BATCH_SIZE, 
    num_workers=14,
    pin_memory=True,
    persistent_workers=False,
    prefetch_factor=8,
)

n_train_batches = len(train_loader)
print(f"[debug] train_loader batches = {n_train_batches}")
assert n_train_batches > 0, "Empty train dataloader. Check min_prediction_idx/ENC_LEN/date windows."

val_loader = val_ds.to_dataloader(
    train=False,
    batch_size=BATCH_SIZE,
    num_workers=14,
    pin_memory=True,
    persistent_workers=False,
    prefetch_factor=8,
)

n_val_batches = len(val_loader)
print(f"[debug] val_loader batches = {n_val_batches}")
assert n_val_batches > 0, "Empty val dataloader. Check min_prediction_idx/ENC_LEN/date windows."

In [ ]:
lp.seed_everything(42)
trainer = lp.Trainer(
    accelerator="gpu",
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)

tft = TemporalFusionTransformer.from_dataset(
    train_ds,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=LR,
    hidden_size=HIDDEN,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=HEADS,
    dropout=DROPOUT,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=HIDDEN // 2,  # set to <= hidden_size
    loss=RMSE(),
    optimizer=torch.optim.Adam,
    # reduce learning rate if no improvement in validation loss after x epochs
    # reduce_on_plateau_patience=1000,
)
print(f"Number of parameters in network: {tft.size() / 1e3:.1f}k")
    
# find optimal learning rate
from lightning.pytorch.tuner import Tuner

res = Tuner(trainer).lr_find(
    tft,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

## main

In [ ]:
lp.seed_everything(42) 

# ========== 8) 训练（按 CV 折） ========== 先取第一折
best_ckpt_paths, fold_metrics = [], []

#for fold_id, (train_days, val_days) in enumerate(folds_by_day, start=1):
####################################
fold_id = 0
train_days, val_days = folds_by_day[fold_id]
####################################

print(f"[fold {fold_id}] train {train_days[0]}..{train_days[-1]} ({len(train_days)} days), "
    f"val {val_days[0]}..{val_days[-1]} ({len(val_days)} days)")

# 明确日期：
train_start_date = int(train_days[0])
train_end_date   = int(train_days[-1])
val_start_date   = int(val_days[0])
val_end_date     = int(val_days[-1])      

# 提取数据
date_range = (train_start_date, val_end_date)
pdf_data = (
    lf_clean
    .filter(pl.col(G_DATE).is_between(train_start_date, val_end_date, closed="both"))
    .collect(streaming=True)
    .to_pandas()
) 

pdf_data[G_SYM] = pdf_data[G_SYM].astype("str")
pdf_data["time_bucket"] = pdf_data["time_bucket"].astype("str")
pdf_data.sort_values([G_SYM, "time_idx"], inplace=True)

In [ ]:

# 明确 indexes:
train_end_idx = pdf_data.loc[pdf_data[G_DATE] == train_end_date, "time_idx"].max()
val_start_idx = pdf_data.loc[pdf_data[G_DATE] == val_start_date, "time_idx"].min()
val_end_idx   = pdf_data.loc[pdf_data[G_DATE] == val_end_date, "time_idx"].max()
assert pd.notna(train_end_idx) and pd.notna(val_start_idx) and pd.notna(val_end_idx), "train/val idx not found"
train_end_idx, val_start_idx, val_end_idx = int(train_end_idx), int(val_start_idx), int(val_end_idx)
print(f"[fold {fold_id}] train idx up to {train_end_idx}, val idx {val_start_idx}..{val_end_idx}")  


In [ ]:
# 构建训练集 timeseries dataset

t_data = pdf_data[TRAIN_COLS]
train_df = t_data.loc[t_data["time_idx"] <= train_end_idx]

train_ds = TimeSeriesDataSet(
    train_df,
    time_idx="time_idx",
    target=TARGET_COL,
    group_ids=[G_SYM],
    weight=WEIGHT_COL,
    max_encoder_length=ENC_LEN, min_encoder_length=ENC_LEN,
    max_prediction_length=PRED_LEN, min_prediction_length=PRED_LEN,
    time_varying_known_reals =KNOWN_REALS,
    time_varying_known_categoricals=KNOWN_CATEGORIES,
    static_categoricals=[G_SYM],
    categorical_encoders={G_SYM: NaNLabelEncoder(add_nan=True),
                        "time_bucket": NaNLabelEncoder(add_nan=True),},
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
    target_normalizer=GroupNormalizer(
        method="standard", groups=[G_SYM], center=True, scale_by_group=False),
    scalers=identity_scalers,
)

In [ ]:

# 验证集复用 train_ds 的所有 encoders/normalizer（不泄漏）

val_df = t_data.loc[t_data["time_idx"].between(val_start_idx, val_end_idx, inclusive="both")]
val_ds = TimeSeriesDataSet.from_dataset(
    train_ds,
    val_df,
    min_prediction_idx=val_start_idx+ENC_LEN+PRED_LEN-1,
    stop_randomization=True,
    predict=True
)

In [ ]:
# 数据集加载

train_loader = train_ds.to_dataloader(
    train=True, 
    batch_size=BATCH_SIZE, 
    num_workers=14,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4,
)

n_train_batches = len(train_loader)
print(f"[debug] train_loader batches = {n_train_batches}")
assert n_train_batches > 0, "Empty train dataloader. Check min_prediction_idx/ENC_LEN/date windows."

val_loader = val_ds.to_dataloader(
    train=False,
    batch_size=BATCH_SIZE*4,
    num_workers=14,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4,
)

n_val_batches = len(val_loader)
print(f"[debug] val_loader batches = {n_val_batches}")
assert n_val_batches > 0, "Empty val dataloader. Check min_prediction_idx/ENC_LEN/date windows."

In [34]:
# 8.6 callbacks/logger/trainer
ckpt_dir_fold = Path(CKPTS_DIR) / f"fold_{fold_id}"
ckpt_dir_fold.mkdir(parents=True, exist_ok=True)

callbacks = [EarlyStopping(monitor="val_loss", mode="min", patience=2, check_on_train_epoch_end=False),
            ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, dirpath=ckpt_dir_fold.as_posix(), filename=f"fold{fold_id}-tft-best-{{epoch:02d}}-{{val_loss:.5f}}", save_on_train_epoch_end=False),
            LearningRateMonitor(logging_interval="step"),]
RUN_NAME = (f"f{fold_id}"f"_E{MAX_EPOCHS}"f"_lr{LR}"f"_bs{BATCH_SIZE}"f"_enc{ENC_LEN}_dec{DEC_LEN}"f"_{datetime.now().strftime('%Y%m%d-%H%M%S')}")
logger = TensorBoardLogger(save_dir=LOGS_DIR.as_posix(),name="tft",version=RUN_NAME,default_hp_metric=False)

trainer = lp.Trainer(max_epochs=MAX_EPOCHS,
                    accelerator="gpu",
                    precision="bf16-mixed",
                    enable_model_summary=True,
                    gradient_clip_val=1.0,
                    #fast_dev_run=True,
                    #limit_train_batches=300,  # coment in for training, running valiation every 30 batches
                    log_every_n_steps=200,
                    callbacks=callbacks,
                    logger=logger,
                    accumulate_grad_batches=4
                    )

tft = TemporalFusionTransformer.from_dataset(
    train_ds,
    learning_rate=LR,
    hidden_size=HIDDEN,
    attention_head_size=HEADS,
    dropout=DROPOUT,
    hidden_continuous_size=HIDDEN // 2,
    loss=RMSE(),
    logging_metrics=[RMSE()],
    optimizer=torch.optim.AdamW,
    optimizer_params={"weight_decay": 1e-4},
    reduce_on_plateau_patience=4,
)
trainer.fit(
    tft,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    )

SystemExit: 1

In [ ]:
ckpt_cb = next(cb for cb in callbacks if isinstance(cb, ModelCheckpoint))
best_path = ckpt_cb.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_path)

predictions = best_tft.predict(
    val_loader,
    return_y=True,
    trainer_kwargs=dict(accelerator="gpu")
)
y_pred = predictions.output
y_true, w = predictions.y

In [ ]:
num = (w * (y_true - y_pred).pow(2)).sum()
den = (w * y_true.pow(2)).sum()

wr2 = 1.0 - num / (den + eps)
print(f"wr2 after training: {wr2.item():.6f}")

In [ ]:
val_ds.get_parameters

In [ ]:
num = (torch.square(y_true - y_pred) * w).sum()
den = (torch.square(y_true) * w).sum()  
wr2 = 1 - num / den
print(f"wr2 after training: {wr2}")